In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

user_id = vexa.user_id
user_name = vexa.user_name
import asyncio
from vector_search import VectorSearch,build_context_string
from typing import Optional, Union, List

User information retrieved successfully.


In [3]:
user_id

'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'

In [4]:
vector_search = VectorSearch()

In [27]:
summaries = await vector_search.get_summaries(user_id=user_id)

In [5]:
from datetime import datetime,timedelta
from qdrant_client import models
from typing import Dict, Any, Tuple
from pydantic import BaseModel, Field
from core import BaseCall
from core import user_msg,assistant_msg,system_msg

In [23]:
class SearchDocumentsInput(BaseCall):
    vector_search_query: Optional[str] = Field(None, description="Query string for vector search")
    start: Optional[Union[str, datetime]] = Field(None, description="Start date/time for filtering")
    end: Optional[Union[str, datetime]] = Field(None, description="End date/time for filtering")
    speakers: Optional[List[str]] = Field(None, description="List of speakers for filtering")

    async def search_documents(self, vector_search: VectorSearch=vector_search,
                               user_id: Optional[str] = user_id, 
                               user_name: Optional[str] = user_name,
                               k: int = 10,
                               include_summary: bool = False) -> List[Tuple[Dict[str, Any], float, str]]:


        filter_conditions = []
        
        if not include_summary:
            filter_conditions.append(models.FieldCondition(key="type", match=models.MatchValue(value="point")))
        
        if self.start:
            adjusted_start = self.start - timedelta(days=1) if isinstance(self.start, datetime) else datetime.fromisoformat(self.start) - timedelta(days=1)
            filter_conditions.append(models.FieldCondition(
                key="start_datetime",
                range=models.Range(gte=int(adjusted_start.timestamp()))
            ))
        if self.end:
            adjusted_end = self.end + timedelta(days=1) if isinstance(self.end, datetime) else datetime.fromisoformat(self.end) + timedelta(days=1)
            filter_conditions.append(models.FieldCondition(
                key="start_datetime",
                range=models.Range(lte=int(adjusted_end.timestamp()))
            ))
        
        if user_id:
            if isinstance(user_id, str):
                filter_conditions.append(models.FieldCondition(key="user_id", match=models.MatchValue(value=user_id)))
            else:
                filter_conditions.append(models.FieldCondition(key="user_id", match=models.MatchAny(any=user_id)))
        
        if user_name:
            if isinstance(user_name, str):
                filter_conditions.append(models.FieldCondition(key="user_name", match=models.MatchValue(value=user_name)))
            else:
                filter_conditions.append(models.FieldCondition(key="user_name", match=models.MatchAny(any=user_name)))

        if self.speakers:
            filter_conditions.append(models.FieldCondition(key="speakers", match=models.MatchAny(any=self.speakers)))

        if self.vector_search_query:
            query_vector = await vector_search.get_embeddings([self.vector_search_query])
            search_result = await vector_search.qdrant_client.search(
                collection_name=vector_search.collection_name,
                query_vector=query_vector[0].tolist(),
                query_filter=models.Filter(must=filter_conditions),
                limit=k
            )
        else:
            search_result = await vector_search.qdrant_client.scroll(
                collection_name=vector_search.collection_name,
                scroll_filter=models.Filter(must=filter_conditions),
                limit=k,
                with_payload=True,
                with_vectors=True
            )
            search_result = search_result[0]  # Unpack the tuple returned by scroll

        return [(hit.payload, getattr(hit, 'score', 1.0), hit.id) for hit in search_result]

In [30]:
general_context = build_context_string(summaries, only_summaries=True)[0]
general_context

'# [1] Marketing Strategy and Content Planning (2024-08-30 13:08) participants: [\'Dmitriy Grankin\', \'Olga Nemirovskaya\']\nDmitriy Grankin and Olga Nemirovskaya discuss marketing strategies, focusing on Google Ads, influencer outreach, and content creation for social media platforms. They plan to enhance engagement through targeted posts and workshops, while also analyzing audience interactions to improve conversion rates.\n\n---\n\n# [2] Marketing Strategy and Content Planning (2024-08-30 13:08) participants: [\'Dmitriy Grankin\', \'Olga Nemirovskaya\']\nOn August 30, 2024, Dmitriy Grankin and Olga Nemirovskaya discussed marketing strategies, focusing on Google Ads, influencer outreach, and content creation for social media platforms. They brainstormed ideas for posts, analyzed audience engagement, and planned workshops to enhance outreach efforts. Key tasks included creating Instagram content, TikTok scripts, and LinkedIn posts to improve audience interaction and conversion rates.

In [31]:
r = await SearchDocumentsInput.call([user_msg('marketing tools')])
r

(SearchDocumentsInput(vector_search_query='marketing tools', start=None, end=None, speakers=None),
 ChatCompletion(id='chatcmpl-ALFW0FbgrLXoqwoKRQw0tFiP10qbO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_m7lvHQi3NeUA89wYct7xG2mO', function=Function(arguments='{"vector_search_query":"marketing tools"}', name='SearchDocumentsInput'), type='function')]))], created=1729627892, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_482c22a7bc', usage=CompletionUsage(completion_tokens=8, prompt_tokens=177, total_tokens=185, completion_tokens_details=None, prompt_tokens_details=None)))

In [25]:
await r[0].search_documents()

[]

In [11]:
self.search_documents = search_documents

In [32]:
await self.search_documents(self,vector_search_query='SEO')

NameError: name 'self' is not defined